# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its outputs are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

(...So the conventional disciplinary thinking goes, at least. But notice that we are doing 120 qubits of Simon's algorithm below, and we could argue we only need a single query, with PyQrack!)

In [1]:
import random
from collections import Counter
from pyqrack import QrackSimulator

# Range is up to 2 ** input_size - 1
input_size = 60

# Arbitrary value within range:
hidden_bits = 2**30 + 1923

num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

# Swapping the bits randomly doesn't seem to work, for now:
oracle_swap_tuples = []
for i in range(input_size // 2):
    i = random.randint(input_size, (input_size * 2) - 1)
    j = random.randint(input_size, (input_size * 2) - 1)
    if i != j:
        oracle_swap_tuples.append((i, j))

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        # We pick any one of the "set" hidden bits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                # Act CNOT between the "set" hidden bit in input and all hidden bits in output:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            # One bit in the "i" loop is sufficient:
            break;

    # We can now swap at random in the output register
    # and still satisfy our promises about the oracle:
    for i in oracle_swap_tuples:
         sim.swap(i[0], i[1])

This is the algorithm itself. (A classical simulator of quantum computers can run the "unitary preamble" exactly once and then secondarily "sample" the measurement distribution of the end state.)

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
results = Counter(sim.measure_shots(oracle_qubits, 32768))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir


The ultimate point of the algorithm is that, by solving the system of equations below, **where the value of `hidden_string` is an algebraic unknown**, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).

Let's check if our implied system of equations is right:

In [4]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    if x > 0:
        print("Failed!", hidden_bits, ".", key, "=", x, "(mod 2)")
        break
    print(hidden_bits, ".", key, "=", x, "(mod 2)")

1073743747 . 994599497 = 0 (mod 2)
1073743747 . 2062720185 = 0 (mod 2)
1073743747 . 2608751991 = 0 (mod 2)
1073743747 . 2273728612 = 0 (mod 2)
1073743747 . 2194979178 = 0 (mod 2)
1073743747 . 1612274994 = 0 (mod 2)
1073743747 . 2804522987 = 0 (mod 2)
1073743747 . 931082695 = 0 (mod 2)
1073743747 . 3631472507 = 0 (mod 2)
1073743747 . 2347420843 = 0 (mod 2)
1073743747 . 1097289834 = 0 (mod 2)
1073743747 . 3660465694 = 0 (mod 2)
1073743747 . 2808125323 = 0 (mod 2)
1073743747 . 2032058076 = 0 (mod 2)
1073743747 . 3711908677 = 0 (mod 2)
1073743747 . 982134272 = 0 (mod 2)
1073743747 . 3124799460 = 0 (mod 2)
1073743747 . 3135844784 = 0 (mod 2)
1073743747 . 3835705338 = 0 (mod 2)
1073743747 . 3931031945 = 0 (mod 2)
1073743747 . 2912630819 = 0 (mod 2)
1073743747 . 3716857247 = 0 (mod 2)
1073743747 . 2798487080 = 0 (mod 2)
1073743747 . 997323678 = 0 (mod 2)
1073743747 . 983310950 = 0 (mod 2)
1073743747 . 1441406627 = 0 (mod 2)
1073743747 . 3418968006 = 0 (mod 2)
1073743747 . 1290579977 = 0 (mod 

1073743747 . 4043273453 = 0 (mod 2)
1073743747 . 4059165330 = 0 (mod 2)
1073743747 . 3833516244 = 0 (mod 2)
1073743747 . 3372129768 = 0 (mod 2)
1073743747 . 3420141031 = 0 (mod 2)
1073743747 . 3990653503 = 0 (mod 2)
1073743747 . 1998917824 = 0 (mod 2)
1073743747 . 100186923 = 0 (mod 2)
1073743747 . 790868342 = 0 (mod 2)
1073743747 . 671406576 = 0 (mod 2)
1073743747 . 3706304731 = 0 (mod 2)
1073743747 . 2137643176 = 0 (mod 2)
1073743747 . 4206043110 = 0 (mod 2)
1073743747 . 3055375575 = 0 (mod 2)
1073743747 . 3526973337 = 0 (mod 2)
1073743747 . 3647517973 = 0 (mod 2)
1073743747 . 1128220039 = 0 (mod 2)
1073743747 . 3961659188 = 0 (mod 2)
1073743747 . 67687330 = 0 (mod 2)
1073743747 . 213554673 = 0 (mod 2)
1073743747 . 874062146 = 0 (mod 2)
1073743747 . 306386698 = 0 (mod 2)
1073743747 . 2322493598 = 0 (mod 2)
1073743747 . 3507345180 = 0 (mod 2)
1073743747 . 3380906888 = 0 (mod 2)
1073743747 . 39164339 = 0 (mod 2)
1073743747 . 2752188130 = 0 (mod 2)
1073743747 . 521393653 = 0 (mod 2)
107